In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

def inception_score(imgs, cuda, batch_size, resize, splits):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)
    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)
    print(batch_size)
    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval()
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x): 

        x = up(x)
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch[0]
#         print(batch)
#         batch = np.array(batch)
#         batch = torch.from_numpy(batch)

        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)

In [2]:
import pandas as pd
import numpy as np
import pickle as pkl

haha=pd.read_csv('data1/fake6.csv')
# weather = pd.read_csv('data/jfk_weather_cleaned.csv')
# haha=weather.drop(columns = ['DATE','Precip','SeaLevelPressure']).astype(float)
haha.info()

k=0
X = np.array([],dtype=float)
for i in range(75000):
    if i%24 == 1:
        xixi=haha.iloc[i:i+24]
        if X.shape[0] == 0:
            X = np.array([xixi.values], dtype=float)
        else:
            X = np.concatenate((X, [xixi.values]))
        #X=np.concatenate((X,xixi.values))
        k+=1
print(k)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DewPointTempF     80000 non-null  float64
 1   DRYBULBTEMPF      80000 non-null  float64
 2   RelativeHumidity  80000 non-null  float64
 3   StationPressure   80000 non-null  float64
 4   VISIBILITY        80000 non-null  float64
 5   WETBULBTEMPF      80000 non-null  float64
 6   WindDirection     80000 non-null  float64
 7   WindSpeed         80000 non-null  float64
dtypes: float64(8)
memory usage: 4.9 MB
3125


In [3]:
pkl.dump(X, open('data1/fake_weather.pkl', 'wb'))

In [4]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import pickle as pkl
import torch.utils.data as data1
# class IgnoreLabelDataset(torch.utils.data.Dataset):
#     def __init__(self, orig):
#         self.orig = orig

#     def __getitem__(self, index):
#         return self.orig[index][0]

#     def __len__(self):
#         return len(self.orig)

data=pkl.load(open('data1/fake_weather.pkl', 'rb'))
# data=np.expand_dims(data, axis=1)


data=np.array([data,data,data])
data=np.reshape(data,(3125,3,24,8))

print(data.shape)

data=torch.from_numpy(data).float()

class MyDataset(data1.Dataset):
    def __init__(self, data):
        
        self.data = data

    def __getitem__(self, index):
       
        return self.data[index],index

    def __len__(self):
        return len(self.data)        

# dataloader = torch.utils.data.DataLoader(dataset=MyDataset(data),batch_size=batch_size,
#                                          shuffle=True,drop_last=True)

# dataloader = torch.utils.data.DataLoader(
#     MyDataset(data),
#     num_workers=0,
#     batch_sampler=UniformWithReplacementSampler(
#         num_samples=len(MyDataset(data)),
#         sample_rate=0.018,
#     ),
# )

# device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

#     cifar = dset.CIFAR10(root='data/', download=True,
#                              transform=transforms.Compose([
#                                  transforms.Scale(32),
#                                  transforms.ToTensor(),
#                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#                              ])
#     )

#     IgnoreLabelDataset(cifar)
dataset=MyDataset(data)


# print(dataset[1])

print ("Calculating Inception Score...")
print (inception_score(dataset, cuda=True, batch_size=32, resize=True, splits=10))

(3125, 3, 24, 8)
Calculating Inception Score...
32


c:\anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
c:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


(2.091944617341215, 0.05146319339189201)
